#### Thinking out loud, at the beginning:
- Wow these files are big, I'm going to need to pare down the data probably a lot.
- First going to be looking for "what's the most complete data that I have for what span of time"?
- What data do I actually want to focus on? (record some research on how, broadly, one comes up with a company valuation that could be comparable to market capitalization)
- I probably should focus my market data gathering on the day after each SEC filing is released, assuming that will be the day that stock price movements are most influenced by the filing information rather than other news or factors.
- My core question so far is, are market prices correlated with valuation information in some fashion?
    - If so, which valuation information is most strongly correlated?
    - What, if any, is the relationship between stock price/market capitalization and the business's balance sheet?
    - Do any of these data include attempts to put a value on intangibles such as goodwill/reputation?
    - The fundamental analysis tradition focuses on ratios of price to earnings, book value, earnings growth; do the ratios that investors find acceptable change over time? Do they tend to change back (or ahead) when SEC filings are released?
    - (Do I have time for this?) What happens in the periods between filings? Do prices maintain a range, like the moving average support/resistance bands favored by technical analysis?
- Therefore, what am I looking for in the data I've found so far?
    - As much history as I can assemble from both market data and SEC filings; will need to drop years where I have one but not the other.
    - Focus on the Fortune 100, probably choose the 100 as of the most recent year I have data for.
    - Market data at daily granularity, hopefully with open, high, low, and close prices for each day. Probably focus on dates related to the company's SEC filings.
    - Still deciding what SEC filing information, partially I don't know what I have to work with yet, hoping for a moderately quick thumbnail of value related to assets, liabilities, and maybe intangibles/goodwill. Assuming earnings-per-share numbers also will be found here.
    - If market data has already calculated P/E or other ratios, great, I can use those to sanity-check what I'm pulling in from SEC.

# First crack at a plan:

## 1. Open each dataset (I have 4 right now) individually, probably not in code. 
- Need to see file structure and years covered. Make decision on how much history to collect/collate.
    - Market datasets from:
        - https://www.kaggle.com/borismarjanovic/price-volume-data-for-all-us-stocks-etfs
            - description does not offer a begin date but last updated in 2017-Nov
            - data appears to be in txt files by ticker: file for AAPL dates to 1984-Sep and columns are Date,Open,High,Low,Close,Volume,OpenInt
            - there is a comment that prices in this data are adjusted for dividends AND splits (as opposed to Yahoo Finance data, which is not or not always--not sure what that means)
        - https://www.kaggle.com/tsaustin/us-historical-stock-prices-with-earnings-data
            - 20 years according to description; last updated 2020-Dec
            - Data in 4 files: stock price, earnings (estimated and reported), dividends, and a summary
    - SEC datasets from:
        - https://www.kaggle.com/miguelaenlle/parsed-sec-10q-filings-since-2006
            - Since 2006 according to description; Last updated 2020-Jul
            - Notes that ~50% of data is null, which might be 'not found in filing' or 'not found by parser'.
            - Does have lots of promising columns like commonstocksharesissued, assetscurrent, accountspayablecurrent, commonstockvalue, liabilities, liabilitiesandstockholdersequity, stockholdersequity, earningspersharebasic, netincomeloss, profitloss, costofgoodssold, filing_date, costsandexpenses, cash 
        - https://www.kaggle.com/finnhub/reported-financials
            - 2010-2020, according to description. Last updated 2021-Mar
            - 4 GB data!
            - Data is in folders by year-qtr; JSON files. Will take some work, I think, to find the tickers I want.
            
So, I can probably get 2010-2020 reasonably complete. Maybe 2006-2020.

In [4]:
import pandas as pd
#import numpy as np
import os
import json

# lots of code will depend on this ticker list
# csv-to-list found at https://stackoverflow.com/questions/21065938/read-a-single-column-of-a-csv-and-store-in-an-array
ticker_list = pd.read_csv('Fortune100.csv', usecols=['Ticker'], na_values='-').dropna().T.values.tolist()[0]
print(ticker_list)
#ticker_list = ['AAPL'] used for original code creation to reduce complexity 

['WMT', 'AMZN', 'XOM', 'AAPL', 'CVS', 'BRKA', 'UNH', 'MCK', 'T', 'ABC', 'GOOGL', 'F', 'CI', 'COST', 'CVX', 'CAH', 'JPM', 'GM', 'WBA', 'VZ', 'MSFT', 'MPC', 'KR', 'FNMA', 'BAC', 'HD', 'PSX', 'CMCS.A', 'ANTM', 'WFC', 'C', 'VLO', 'GE', 'DELL', 'JNJ', 'TGT', 'IBM', 'RTX', 'BA', 'FMCC', 'CNC', 'UPS', 'LOW', 'INTC', 'FB', 'FDX', 'MET', 'DIS', 'PG', 'PEP', 'HUM', 'PRU', 'ADM', 'ACI', 'SYY', 'LMT', 'HPQ', 'ET', 'GS', 'MS', 'CAT', 'CSCO', 'PFE', 'HCA', 'AIG', 'AXP', 'DAL', 'MRK', 'AAL', 'CHTR', 'ALL', 'BBY', 'UAL', 'DOW', 'TSN', 'TJX', 'ORCL', 'GD', 'DE', 'NKE', 'PGR', 'KO', 'TECD', 'INT', 'HON', 'COP', 'EXC', 'NOC', 'COF', 'PAGP', 'ABBV', 'SNEX']


## 2. Open each dataset in code and examine columns.
See if I can choose one market and one SEC dataset; do they contain the same information? Is there just a column or two from one I can add to the other?

### 2a: the Finnhub SEC filings dataset
- Hopefully the most complicated one!

In [5]:
# Note some of this code may not work as written except on my own computer, or if you download the datasets like I did.
# Not planning to upload all this data, it's waaay too much.
# CAUTION: walking all this data will take a long time! Over 200K files, expect 20+ min of "busy" before you get an answer.
# I am grateful that Jupyter changes the tab icon to hourglass when a notebook is busy!

# For Finnhub reported-financials dataset:
# Recurse through many YYYY.Q# subdirectories using os.walk
# Acquire list of files containing the ticker(s) I want (define a list somewhere early!)
# Copy those files to a "data I'm keeping" folder
# Import reduced file set into a dataframe for analysis
# I thought about defining a function, but it's pretty specific to how this one dataset is laid out, is it worth it?


start_dir = os.path.join(os.getcwd(), 'SEC-FinancialsAsReported') 
files_list = []
problem_files = []

for root, subdirs, files in os.walk(start_dir):
    for filename in files:
        if os.path.splitext(filename)[1] == ('.json'):
            file_path = os.path.join(root, filename)
            try:
                df = pd.read_json(file_path) # is there a better way to read json if all I want is to check one column?
            except ValueError as val_err:
                problem_files.append((file_path, "Value Error: {0}".format(val_err)))
            except:
                print("Unexpected error:", sys.exc_info()[0])
                raise
            else:
                set_tickers = set(ticker_list)
                set_symbols = set(df['symbol'].unique())
                if set_tickers.intersection(set_symbols):
                    # print('Found one!')
                    files_list.append(file_path)

print('Files found: ' + str(len(files_list)))
print('Problem files found: ' + str(len(problem_files)))
# Files found with original single ticker: 44, 1 problem file

Files found: 3726
Problem files found: 1


#### experimenting with the one problem file. Set lines? orient? skip because it's one file out of 200K or so?
`
print(os.getcwd())  
file_problem = os.path.join(os.getcwd(),'SEC-FinancialsAsReported\\2016.QTR4\\0001098009-16-000023.json')  
print(file_problem)  
`

tried a lot of variations to get path join to work by hand (as opposed to using walk). 
The two most important seem to be start with os.getcwd() and escape any slashes in the path.  
`
df = pd.read_json(file_problem) #as done in filewalk above, "value is too big"  
df = pd.read_json(file_problem, lines=True) #did not work, "expected object or value"  
df = pd.read_json(file_problem, orient='records') #did not work, "value is too big"  
df = pd.read_json(file_problem, orient='split') #did not work, "value is too big"  
df = pd.read_json(file_problem, orient='index') #did not work, "value is too big"  
df = pd.read_json(file_problem, orient='columns') #did not work, "value is too big"  
df = pd.read_json(file_problem, orient='values') #did not work, "value is too big"  
print(df.head)  
`  
nothing is working so far, skip this damn file. Opened in Notepad++ and its ticker is 'CWNM' 
which I don't think is on my list anyway.

#### Now, to see what I have for my ticker(s) in finnhub files:

In [6]:
# json_normalize found at https://stackoverflow.com/questions/20638006/convert-list-of-dictionaries-to-a-pandas-dataframe
# convert to dict clue found at https://stackoverflow.com/questions/55710154/python-json-normalize-string-indices-must-be-integers-error
# nested JSON clue 1: https://stackoverflow.com/questions/47341519/json-normalize-for-dicts-within-dicts
# nested JSON clue 2: https://stackoverflow.com/questions/46091362/how-to-normalize-json-correctly-by-python-pandas (needs json.load)
# nested JSON clue 3: https://www.kaggle.com/jboysen/quick-tutorial-flatten-nested-json-in-pandas/notebook
bs_df_list = []
cf_df_list = []
ic_df_list = []

for i in range(len(files_list)):
    with open(files_list[i]) as f:
        d = json.load(f)
    balsheet_df = pd.json_normalize(d, record_path=['data','bs'], meta=['startDate','endDate','year','quarter','symbol'])
    cashflow_df = pd.json_normalize(d, record_path=['data','cf'], meta=['startDate','endDate','year','quarter','symbol'])
    income_df = pd.json_normalize(d, record_path=['data','ic'], meta=['startDate','endDate','year','quarter','symbol'])
    bs_df_list.append(balsheet_df)
    cf_df_list.append(cashflow_df)
    ic_df_list.append(income_df)
    
#print(bs_df_list)
bs_df = pd.concat(bs_df_list, ignore_index=True)
cf_df = pd.concat(cf_df_list, ignore_index=True)
ic_df = pd.concat(ic_df_list, ignore_index=True)

print(bs_df.head())
print(bs_df.info())
print(bs_df.describe())
print(bs_df.endDate.unique())
print(bs_df.symbol.unique())
# These 3 dataframes with single ticker held ~1000-1300 records over 44 quarterly filings and occupied 70-90K memory each
# One ticker likely to not be enough data points for good machine learning!

  label                                concept unit       value   startDate  \
0        CashAndCashEquivalentsAtCarryingValue  usd  1614607000  2009-02-01   
1                         ShortTermInvestments  usd   130636000  2009-02-01   
2                 AccountsReceivableNetCurrent  usd   148126000  2009-02-01   
3                                 InventoryNet  usd  2532318000  2009-02-01   
4                           OtherAssetsCurrent  usd   255707000  2009-02-01   

      endDate  year quarter symbol  
0  2010-01-30  2010      FY    TJX  
1  2010-01-30  2010      FY    TJX  
2  2010-01-30  2010      FY    TJX  
3  2010-01-30  2010      FY    TJX  
4  2010-01-30  2010      FY    TJX  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 123738 entries, 0 to 123737
Data columns (total 9 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   label      123738 non-null  object
 1   concept    123738 non-null  object
 2   unit       123738 non-null  

In [7]:
print(cf_df.head())
print(cf_df.info())
print(cf_df.describe())
print(cf_df.endDate.unique())
print(cf_df.symbol.unique())

                                               label  \
0                                                      
1                                                      
2  Loss on property disposals and impairment charges   
3                                                      
4                                                      

                                           concept unit       value  \
0                                       ProfitLoss  usd  1213572000   
1             DepreciationDepletionAndAmortization  usd   435218000   
2  tjx:LossOnPropertyDisposalsAndImpairmentCharges  usd    10270000   
3                  DeferredIncomeTaxExpenseBenefit  usd    53155000   
4                           ShareBasedCompensation  usd    55145000   

    startDate     endDate  year quarter symbol  
0  2009-02-01  2010-01-30  2010      FY    TJX  
1  2009-02-01  2010-01-30  2010      FY    TJX  
2  2009-02-01  2010-01-30  2010      FY    TJX  
3  2009-02-01  2010-01-30  2010      FY 

In [8]:
print(ic_df.head())
print(ic_df.info())
print(ic_df.describe())
print(ic_df.endDate.unique())
print(ic_df.symbol.unique())

                                               label  \
0                                                      
1  Cost of sales, including buying and occupancy ...   
2                                                      
3                              Interest expense, net   
4                                                      

                                             concept unit        value  \
0                                           Revenues  usd  2.02884e+10   
1    tjx:CostOfSalesIncludingBuyingAndOccupancyCosts  usd -1.49684e+10   
2             SellingGeneralAndAdministrativeExpense  usd  3.32894e+09   
3                             tjx:InterestExpenseNet  usd   3.9509e+07   
4  IncomeLossFromContinuingOperationsBeforeIncome...  usd  1.95156e+09   

    startDate     endDate  year quarter symbol  
0  2009-02-01  2010-01-30  2010      FY    TJX  
1  2009-02-01  2010-01-30  2010      FY    TJX  
2  2009-02-01  2010-01-30  2010      FY    TJX  
3  2009-02-01  2010-01

##### Some things I needed to find out:
- Why are all these reports split into records called "bs/cf/ic"? What are those abbreviations for?
- how about... **B**alance **S**heet, **C**ash **F**low, **I**n**C**ome statement? https://www.sec.gov/oiea/reportspubs/investor-publications/beginners-guide-to-financial-statements.html
- I want to see more of what's in those "data" fields: learning how to unpack nested JSON
- Describe doesn't do much when every number is a different thing!

#### Already I'm seeing:
- Not all the tickers are in all 3 dataframes unless they're showing up in different order ('LOW' is one example)
- This might be all the tickers I'm looking for; I count 88 in the income dataframe. Might be not quite. Most likely enough
- There's still one thing that's hard to see here, which is how many of the label and/or concept columns are the same across companies and filings. How sparse will this data be once I figure out the categorization?

***TODO:*** code to save the files to a location I'm willing to upload, code to save the dataframes for later, melt these dataframes and see where the commonalities are (group/sort by label and/or concept)

In [10]:
save_dir = os.path.join(os.getcwd(), 'RawData')
bs_df.to_csv(os.path.join(save_dir, 'FH-balsheets-tickers.csv'))
cf_df.to_csv(os.path.join(save_dir, 'FH-cashflow-tickers.csv'))
ic_df.to_csv(os.path.join(save_dir, 'FH-income-tickers.csv'))

### 2b. The miguelaenlle SEC dataset "Historical Financials"
- Should be far less complicated! Everything in one CSV file

In [9]:
hf_path = os.path.join(os.getcwd(), 'SEC-HistoricalFinancials', 'quarterly_financials.csv')
hf_df = pd.read_csv(hf_path, index_col=0, low_memory=False)
# dtype warning for mixed types in column 43? maybe this is about the first ticker being 'AA$'?
# set low_memory as the warning suggested because I'm only using this DF to look for the tickers I want
hf_tickers_df = hf_df[hf_df['stock'].isin(ticker_list)]
print(hf_tickers_df.info())
print(hf_tickers_df.describe())
print(hf_tickers_df.stock.unique())

# This dataframe with one ticker held 39 records and used ~14K memory

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2743 entries, 11 to 100690
Data columns (total 44 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   commonstocksharesissued           2471 non-null   float64
 1   assetscurrent                     2375 non-null   float64
 2   accountspayablecurrent            2083 non-null   float64
 3   commonstockvalue                  2160 non-null   float64
 4   liabilities                       1751 non-null   float64
 5   liabilitiesandstockholdersequity  2743 non-null   float64
 6   stockholdersequity                2503 non-null   float64
 7   earningspersharebasic             2739 non-null   float64
 8   netincomeloss                     2639 non-null   float64
 9   profitloss                        1906 non-null   float64
 10  costofgoodssold                   1168 non-null   float64
 11  filing_date                       2743 non-null   object 
 12  cos

In [11]:
hf_tickers_df.to_csv(os.path.join(save_dir, 'HF-financials-tickers.csv'))

I see a few entirely blank columns here, get rid of those later.  
How do I want to line up this information with the Finnhub data?

### 2c. The borismarjanovic 'Huge Dataset' of market prices

In [17]:
hd_path = os.path.join(os.getcwd(), 'Market-HugeDataset', 'Stocks')
hd_filelist = [os.path.join(hd_path, t.lower() + '.us.txt') for t in ticker_list]
# print(hd_filelist[:6])    
hd_df_list = []
hd_problem_files = []
for file in hd_filelist:
    try:
        df = pd.read_csv(file)
    except FileNotFoundError as fnf_err:
        hd_problem_files.append(file)
    except:
        print("Unexpected error:", sys.exc_info()[0])
        raise
    else:
        df['ticker'] = str(os.path.split(file)[-1]).split('.')[0]
        #print(df.head())
        hd_df_list.append(df)
# concat/map solution found at https://stackoverflow.com/questions/20906474/import-multiple-csv-files-into-pandas-and-concatenate-into-one-dataframe
#pd.concat(map(lambda f: pd.read_csv(f, keys=os.path.split(f)[-1]), hd_filelist)) nope, there's no keys arg to read_csv
# oops, I'm going to need some way to mark which ticker these go with!!

print('Files found: ' + str(len(hd_df_list)))
print('Problem files found: ' + str(len(hd_problem_files)))
#print(hd_problem_files)
# Okay, so one or two of these problem files is a "maybe" but I'm not sure enough, skip them and go with the matches.
hd_df = pd.concat(hd_df_list, ignore_index=True)
print(hd_df.head())
print(hd_df.info())
print(hd_df.describe())

# This is likely to still be somewhat big. This dataframe with one ticker held 8364 records and used 458K memory.

Files found: 82
Problem files found: 10
         Date     Open     High      Low    Close   Volume  OpenInt ticker
0  1972-03-20  0.03403  0.03403  0.03403  0.03403  3132119        0    wmt
1  1972-03-22  0.03403  0.04251  0.03403  0.03403  1084194        0    wmt
2  1972-03-23  0.03403  0.04251  0.03403  0.03403   783030        0    wmt
3  1972-03-24  0.03403  0.04251  0.03403  0.03403  1264892        0    wmt
4  1972-03-27  0.04251  0.04251  0.04251  0.04251  1385361        0    wmt
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 629798 entries, 0 to 629797
Data columns (total 8 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   Date     629798 non-null  object 
 1   Open     629798 non-null  float64
 2   High     629798 non-null  float64
 3   Low      629798 non-null  float64
 4   Close    629798 non-null  float64
 5   Volume   629798 non-null  int64  
 6   OpenInt  629798 non-null  int64  
 7   ticker   629798 non-null  object 
dtypes: fl

(Notes from original run)

Do I believe these AAPL prices? OK, a quick search says today's price was `$`134.16 at last close. Wonder if I'll see what the effect of adjustment is by comparing with the next dataset? I'm pretty sure I've noticed AAPL's price being higher than `$`175 in my random contacts with stock market reports.

(Notes from expanded tickers)

Wow, Walmart goes back to 1972 and prices were 3 cents a share? I really don't know if I believe these prices. At minimum I want to know a lot more about how he "adjusted" them. Was inflation accounted for? 

In [18]:
hd_df.to_csv(os.path.join(save_dir, 'HD-market-tickers.csv'))

### 2d. The tsaustin 'Historical Prices' market dataset

In [19]:
hp_filepath = os.path.join(os.getcwd(), 'Market-HistoricalPrices')
hp_df_s = pd.read_csv(os.path.join(hp_filepath, 'dataset_summary.csv'))
hp_df_tckr_s = hp_df_s[hp_df_s['symbol'].isin(ticker_list)]
print(hp_df_tckr_s.head())
print(hp_df_tckr_s.info())
hp_df_e = pd.read_csv(os.path.join(hp_filepath, 'stocks_latest', 'earnings_latest.csv'))
hp_df_tckr_e = hp_df_e[hp_df_e['symbol'].isin(ticker_list)]
print(hp_df_tckr_e.head())
print(hp_df_tckr_e.info())
hp_df_d = pd.read_csv(os.path.join(hp_filepath, 'stocks_latest', 'dividends_latest.csv'))
hp_df_tckr_d = hp_df_d[hp_df_d['symbol'].isin(ticker_list)]
print(hp_df_tckr_d.head())
print(hp_df_tckr_d.info())
hp_df_p = pd.read_csv(os.path.join(hp_filepath, 'stocks_latest', 'stock_prices_latest.csv'))
hp_df_tckr_p = hp_df_p[hp_df_p['symbol'].isin(ticker_list)]
print(hp_df_tckr_p.head())
print(hp_df_tckr_p.info())
print(hp_df_tckr_p.describe())
# First run with single ticker, 1 entry in summary, 46 entries in earnings, 34 entries in dividends, ~3K memory for all 3 DFs
# Also first run, 5756 entries and 450K memory for price DF

    symbol  total_prices stock_from_date stock_to_date  total_earnings  \
9      AAL          3811      2005-09-27    2020-11-13              28   
16    AAPL          5756      1998-01-02    2020-11-13              46   
27    ABBV          1975      2013-01-02    2020-11-03              31   
28     ABC          5748      1998-01-02    2020-11-03              46   
107    ADM          5750      1998-01-02    2020-11-05              47   

    earnings_from_date earnings_to_date  
9           2014-01-28       2020-10-22  
16          2009-07-21       2020-10-29  
27          2013-04-26       2020-10-30  
28          2009-07-30       2020-11-05  
107         2009-05-05       2020-10-29  
<class 'pandas.core.frame.DataFrame'>
Int64Index: 86 entries, 9 to 7675
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   symbol              86 non-null     object
 1   total_prices        86 non-null     int64 
 2   st

(Notes from first run)

See? This one suggests much higher max prices. What's going on here?

In [20]:
hp_df_tckr_s.to_csv(os.path.join(save_dir, 'HP-market-ticker-summary.csv'))
hp_df_tckr_e.to_csv(os.path.join(save_dir, 'HP-market-ticker-earnings.csv'))
hp_df_tckr_d.to_csv(os.path.join(save_dir, 'HP-market-ticker-dividends.csv'))
hp_df_tckr_p.to_csv(os.path.join(save_dir, 'HP-market-ticker-prices.csv'))

## 3. Choose 100 companies and pull their data into new files to reduce data sizes. 
Let's try the 2020 Fortune 100.
Data retrieved from https://fortune.com/fortune500/2020/search/?rank=asc
Spent a bunch of time massaging the text as copied back into tabular format. Spent a bunch more time going through each company's profile page to pull out ticker symbols. Hopefully an employer would have a better way to access the data, but it's good enough for this project. They want \$500 or more for a data file and don't talk about having an API that I can see.

- Some ticker notes:
    - There are a bunch of companies (especially mutual insurance companies) without tickers, so I'm going to end up with less than 100 companies. Expand search or leave it?
    - Will I need to look out for name changes (esp ticker changes)?
    - Albertsons has a ticker but IPO was last year sometime so may not have data

### This was done by adjusting code so it can all iterate over a list of tickers and pull out selected data.

## 4. Begin EDA, look for cleaning needs. 
- Figure out how best to pull only the SEC-related dates of market data, assuming that dates will generally be different for each company.
- Calculate (can I sanity-check somehow?) fundamental analysis ratios: P/E, P/E/G, price/book.

### I am so starting a new damn notebook for the next wrangling steps and EDA.

## Other tasks as I think of them while exploring!
- Do I have data that shows market capitalization? Or shares outstanding, so I can calculate?
- How does one adjust prices for dividends, splits? What price do I need to use to calculate ratios, market cap, etc?

## Questions for later:
- Once I produce something with this, should I share it on Kaggle? If I've combined datasets, can I link my code to both of them?
- Can or should I try to confirm that Yahoo Finance is a bad place to get stock data from? (see notes on first market dataset above)
- How do I pull out the data I want from these JSON files and save for later? As dataframe, csv, something else?
- Can I pull additional company data from the Fortune site? Should I? https://fortune.com/company/stonex-group/fortune500/

### Research:
- https://www.investopedia.com/terms/a/adjusted_closing_price.asp
- https://www.investopedia.com/articles/fundamental-analysis/08/sec-forms.asp